In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/customer-analytics/Train.csv")
df

In [ ]:
##cleaning null data
df = df.dropna()
df

In [ ]:
##encode label menjadi numerik

encoder = LabelEncoder()
df['Mode_of_Shipment'] = encoder.fit_transform(df['Mode_of_Shipment'])
df['Product_importance'] = encoder.fit_transform(df['Product_importance'])
df['Warehouse_block'] = encoder.fit_transform(df['Warehouse_block'])
df['Gender'] = encoder.fit_transform(df['Gender'])
df

In [ ]:
## perbandingan isian dari target

tarr = df['Reached.on.Time_Y.N'].value_counts()
plt.bar(tarr.index, tarr.values)
plt.show()

In [ ]:
## korelasi antar variable

corr = df.corr().round(3)
plt.figure(figsize=(15,12))
sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
##drop column that 20% correlated because we will use naive bayes that need 'naive' variable
df = df.drop(['Discount_offered', 'Weight_in_gms'], axis=1)
df

In [ ]:
## choosing feature and target

X = df.drop('Reached.on.Time_Y.N', axis=1)
y = df['Reached.on.Time_Y.N']

In [ ]:
## split data with model_selection sklearn

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

In [ ]:
## train data with GaussianNB

nb = GaussianNB()
nb = nb.fit(X_train, y_train)
nb.get_params()

In [ ]:
## confusion matrix
confusion_matrix = plot_confusion_matrix(nb, X_test, y_test)

In [ ]:
## melakukan prediksi dan melihat skor akurasi
y_pred = nb.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
## with isotonic calibration

clf_isotonic = CalibratedClassifierCV(nb, cv='prefit', method='isotonic')
clf_isotonic.fit(X_train, y_train)

y_pred = clf_isotonic.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
## with sigmoid calibration

clf_sigmoid = CalibratedClassifierCV(nb, cv='prefit', method='sigmoid')
clf_sigmoid.fit(X_train, y_train)

y_pred = clf_sigmoid.predict(X_test)
accuracy_score(y_test, y_pred)